In [ ]:
import yfinance as yf
import pandas as pd
import os
import glob
from datetime import datetime
import numpy as np

In [ ]:
path = os.getcwd()

In [ ]:
# Get all CSV files with 'stock_data' in filename
stock_files = glob.glob("*stock_data*.csv")

print("Stock data files:")
for file in stock_files:
    print(f"  {file}")

In [ ]:
file_name = "stock_data_current_20250906_012221.csv"
file_path = path + "/" + file_name

In [ ]:
df = pd.read_csv(file_path, header = 0)
df = df.rename(columns={"Unnamed: 0": "Metrics"})
df = df.set_index('Metrics')

In [ ]:
df

In [ ]:
def filter_stocks_by_criteria(df, criteria):
    """
    Filter stocks based on multiple criteria
    
    Parameters:
    df: DataFrame from get_stock_metrics()
    criteria: dict of filtering criteria, e.g.:
        {
            'P/E Ratio TTM': {'max': 15, 'min': 1},
            'Current Ratio MRQ': {'min': 1.5},
            'ROE TTM': {'min': 0.15},
            'Debt to Equity MRQ': {'max': 50}
        }
    
    Returns:
    dict: {
        'filtered_tickers': list of tickers that meet all criteria,
        'results_summary': summary stats,
        'detailed_results': DataFrame with only filtered tickers
    }
    """
    filtered_tickers = []
    filter_results = {}
    
    for ticker in df.columns:
        meets_all_criteria = True
        ticker_results = {'ticker': ticker}
        
        # Check each criterion
        for metric, conditions in criteria.items():
            if metric not in df.index:
                ticker_results[f"{metric}_status"] = "metric_not_found"
                meets_all_criteria = False
                continue
                
            value = df.loc[metric, ticker]
            
            # Skip if no data (including NaN values)
            if pd.isna(value) or value == 'N/A' or value == 'NaN':
                ticker_results[f"{metric}_status"] = "no_data"
                meets_all_criteria = False
                continue
            
            # Convert to float if it's a string number
            try:
                if isinstance(value, str):
                    value = float(value)
            except (ValueError, TypeError):
                ticker_results[f"{metric}_status"] = "conversion_error"
                meets_all_criteria = False
                continue
            
            # Final check for numeric validity
            if not isinstance(value, (int, float)) or np.isnan(value):
                ticker_results[f"{metric}_status"] = "invalid_data"
                meets_all_criteria = False
                continue
                
            ticker_results[f"{metric}_value"] = value
            
            # Check min condition
            if 'min' in conditions and value < conditions['min']:
                ticker_results[f"{metric}_status"] = f"below_min_{conditions['min']}"
                meets_all_criteria = False
                continue
                
            # Check max condition  
            if 'max' in conditions and value > conditions['max']:
                ticker_results[f"{metric}_status"] = f"above_max_{conditions['max']}"
                meets_all_criteria = False
                continue
                
            ticker_results[f"{metric}_status"] = "pass"
        
        filter_results[ticker] = ticker_results
        
        if meets_all_criteria:
            filtered_tickers.append(ticker)
    
    # Create filtered DataFrame
    filtered_df = df[filtered_tickers] if filtered_tickers else pd.DataFrame()
    
    # Summary stats
    summary = {
        'total_tickers_analyzed': len(df.columns),
        'tickers_meeting_criteria': len(filtered_tickers),
        'success_rate': (len(filtered_tickers) / len(df.columns)) * 100 if df.columns.size > 0 else 0,
        'criteria_applied': criteria
    }
    
    return {
        'filtered_tickers': filtered_tickers,
        'results_summary': summary,
        'detailed_results': filtered_df,
        'filter_breakdown': filter_results
    }

In [ ]:
def get_good_pe_stocks(df, max_pe=15):
    """
    Simple wrapper to get stocks with good P/E ratios
    
    Parameters:
    df: DataFrame from get_stock_metrics()
    max_pe: maximum P/E ratio (default: 15)
    
    Returns:
    dict: Results from filter_stocks_by_criteria
    """
    criteria = {
        'P/E Ratio TTM': {'max': max_pe, 'min': 1}
    }
    
    return filter_stocks_by_criteria(df, criteria)

In [ ]:
# Filter for good P/E ratios (15 or lower)
print("\n🎯 FILTERING FOR GOOD P/E RATIOS:")
pe_results = get_good_pe_stocks(df, max_pe=15)

print(f"✅ Found {len(pe_results['filtered_tickers'])} stocks with P/E ≤ 15:")
for ticker in pe_results['filtered_tickers']:
    pe_value = df.loc['P/E Ratio TTM', ticker]
    company = df.loc['Company Name', ticker]
    print(f"  • {ticker}: {company} (P/E: {pe_value})")

print(f"\n📊 Success Rate: {pe_results['results_summary']['success_rate']:.1f}%")


In [ ]:
def get_value_stocks(df):
    """
    Get stocks that meet multiple value investing criteria
    
    Parameters:
    df: DataFrame from get_stock_metrics()
    
    Returns:
    dict: Results from filter_stocks_by_criteria
    """
    criteria = {
        'P/E Ratio TTM': {'max': 15, 'min': 5},
        'P/B Ratio': {'max': 3.3, 'min': 0.3},
        'P/S Ratio TTM': {'max': 3, 'min': 0.1},
        'ROE TTM': {'min': 0.12},
        'Current Ratio MRQ': {'min': 1.2},
        'Debt to Equity MRQ': {'max': 75}, 
    }
    
    return filter_stocks_by_criteria(df, criteria)

In [ ]:
# Example: Filter for value stocks with multiple criteria
print("\n💎 FILTERING FOR VALUE STOCKS:")
value_results = get_value_stocks(df)

print(f"✅ Found {len(value_results['filtered_tickers'])} value stocks:")
for ticker in value_results['filtered_tickers']:
    pe_value = df.loc['P/E Ratio TTM', ticker] 
    pb_value = df.loc['P/B Ratio', ticker]
    ps_value = df.loc['P/S Ratio TTM', ticker]
    roe_value = df.loc['ROE TTM', ticker]
    current_ratio = df.loc['Current Ratio MRQ', ticker]
    debt_to_equity = df.loc['Debt to Equity MRQ', ticker]
    company = df.loc['Company Name', ticker]
    print(f"  • {ticker}: {company}")
    print(f"    P/E: {pe_value}")
    print(f"    P/B: {pb_value}")
    print(f"    P/S: {ps_value}")
    print(f"    ROE: {roe_value}")
    print(f"    Current Ratio: {current_ratio}")
    print(f"    Debt/Equity: {debt_to_equity}")
    
print(f"\n📊 Success Rate: {value_results['results_summary']['success_rate']:.1f}%")

In [ ]:
# Save filtered results
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# if pe_results['filtered_tickers']:
#     pe_filename = f'good_pe_stocks_{timestamp}.csv'
#     pe_results['detailed_results'].to_csv(pe_filename)
#     print(f"\n💾 Good P/E stocks saved to: {pe_filename}")
